# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [2]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [11]:
#your code here
spaceship.dropna(inplace=True)
spaceship['Cabin'] = spaceship['Cabin'].str.split('/').str[0]
spaceship = spaceship.drop(columns = ['PassengerId','Name'])


features = spaceship.drop(columns = ['Transported'])
target = spaceship['Transported']
spaceship.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


**Perform Train Test Split**

In [57]:
#your code here
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

#adjust numeric columns & categorical columns before doing the normalizer
X_train_numb_columns = X_train.select_dtypes(include=['number'])
X_test_numb_columns = X_test.select_dtypes(include=['number'])

#non numerical columns
X_train_non_numb_columns = X_train.select_dtypes(include=['object'])
X_test_non_numb_columns = X_test.select_dtypes(include=['object'])

In [89]:
#Create a normalizer
normalizer = MinMaxScaler()
normalizer.fit(X_train_numb_columns)
X_train_norm = normalizer.transform(X_train_numb_columns)
X_test_norm = normalizer.transform(X_test_numb_columns)

#Onehotencoder for nominal categorical columns.
ohe = OneHotEncoder(sparse_output=False)

#Transforming data for categorical columns
ohe.fit(X_train_non_numb_columns)
X_train_cat = ohe.transform(X_train_non_numb_columns)
X_test_cat = ohe.transform(X_test_non_numb_columns)

#convert numerical data back to DataFrame
X_train_norm_df = pd.DataFrame(X_train_norm, columns=X_train_numb_columns.columns)
X_test_norm_df = pd.DataFrame(X_test_norm, columns=X_test_numb_columns.columns)

# Convert one-hot encoded categorical data back to DataFrame
X_train_cat_df = pd.DataFrame(X_train_cat, columns=ohe.get_feature_names_out(X_train_non_numb_columns.columns))
X_test_cat_df = pd.DataFrame(X_test_cat, columns=ohe.get_feature_names_out(X_test_non_numb_columns.columns))

# Concatenate numeric and categorical DataFrames
X_train_final = pd.concat([X_train_norm_df, X_train_cat_df], axis=1)
X_test_final = pd.concat([X_test_norm_df, X_test_cat_df], axis=1)

# Display shape of final datasets
X_train_final.shape, X_test_final.shape

((5284, 24), (1322, 24))

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [133]:
#your code here
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


bagging_clf = BaggingClassifier(DecisionTreeClassifier(),
                               n_estimators=100, # number of models to use
                               max_samples = 1000)

bagging_clf.fit(X_train_final, y_train)
y_pred_test_bag = bagging_clf.predict(X_test_final)

print(f"R2 score {bagging_clf.score(X_test_final, y_test): .2f}")

R2 score  0.79


- Random Forests

In [137]:
#your code here

forest_clf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=0)
forest_clf.fit(X_train_final, y_train)

print(f"R2 score, {forest_clf.score(X_test_final, y_test): .2f}")

R2 score,  0.80


- Gradient Boosting

In [141]:
#your code here
gb_clf = GradientBoostingClassifier(max_depth=20, n_estimators=100, random_state=0)
gb_clf.fit(X_train_final, y_train)
                                   
print(f"R2 score, {gb_clf.score(X_test_final, y_test): .2f}")

R2 score,  0.79


- Adaptive Boosting

In [143]:
#your code here

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=100, random_state=0)
ada_clf.fit(X_train_final, y_train)

print(f"R2 score, {ada_clf.score(X_test_final, y_test): .2f}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


R2 score,  0.78


Which model is the best and why?

In [ ]:
#comment here

#The random forests model is throwing the largest R2 score so I'd pick that one.